In [2]:
import geopandas as gpd

# Load the GeoJSON files
local_roads = gpd.read_file('/home/kaiwen1/30123-Project-KaiwenCongo-local-roads.geojson')
main_roads = gpd.read_file('/home/kaiwen1/30123-Project-KaiwenCongo-main-roads.geojson')
villages = gpd.read_file('/home/kaiwen1/30123-Project-KaiwenCongo-villages.geojson')

# Load the shapefiles from the folders
protected_areas = gpd.read_file('/home/kaiwen1/30123-Project-KaiwenCongo-protected/protected_areas.shp')
waterways = gpd.read_file('/home/kaiwen1/30123-Project-KaiwenCongo-water/hotosm_cod_waterways_lines_shp.shp')

# Display the first few rows of each GeoDataFrame to verify loading
print(local_roads.head())
print(main_roads.head())
print(villages.head())
print(protected_areas.head())
print(waterways.head())


ModuleNotFoundError: No module named 'geopandas'

In [3]:
target_crs = "EPSG:4326"  # Common CRS (WGS 84)

# Function to set CRS if it's not set
def set_crs_if_needed(gdf, target_crs):
    if gdf.crs is None:
        gdf.set_crs(target_crs, inplace=True)
    return gdf

# Ensure all GeoDataFrames have the same CRS
local_roads = set_crs_if_needed(local_roads, target_crs)
main_roads = set_crs_if_needed(main_roads, target_crs)
villages = set_crs_if_needed(villages, target_crs)
protected_areas = set_crs_if_needed(protected_areas, target_crs)
waterways = set_crs_if_needed(waterways, target_crs)

# Transform all GeoDataFrames to the target CRS
local_roads = local_roads.to_crs(target_crs)
main_roads = main_roads.to_crs(target_crs)
villages = villages.to_crs(target_crs)
protected_areas = protected_areas.to_crs(target_crs)
waterways = waterways.to_crs(target_crs)

# Display the first few rows of each GeoDataFrame to verify loading
print(local_roads.head())
print(main_roads.head())
print(villages.head())
print(protected_areas.head())
print(waterways.head())

# Display the CRS of each GeoDataFrame to verify
print(local_roads.crs)
print(main_roads.crs)
print(villages.crs)
print(protected_areas.crs)
print(waterways.crs)




              thematic                 mkey osmtype     osm_id       lat  \
0  Highway minor roads         highway=path       w  679880800  2.896716   
1  Highway minor roads  highway=residential       w  680185274  2.930864   
2  Highway minor roads      highway=service       w  427631730 -4.302351   
3  Highway minor roads        highway=track       w  696911276 -5.781419   
4  Highway minor roads        highway=track       w  438149437 -2.934336   

         lon  name   ref      highway tunnel  ... lanes width maxspeed  \
0  30.853455  None  None         path   None  ...  None  None     None   
1  30.313202  None  None  residential   None  ...  None  None     None   
2  15.302207  None  None      service   None  ...  None  None     None   
3  16.122834  None  None        track   None  ...  None  None     None   
4  28.819704  None  None        track   None  ...  None  None     None   

  surface practicability smoothness condition                other_tags  \
0    None           Non

In [4]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from numba import jit
import numpy as np


#Load the CSV file
df_complete = pd.read_csv('/Users/kd6801/Desktop/Mining-Project/NDVI-sample.csv')

In [5]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from numba import jit
import numpy as np
# choose the first 100 elements
df = df_complete[:1000]

# Convert the CSV data to a GeoDataFrame
df['geometry'] = df.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
mining_locations = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:4326")

# Define the target CRS
target_crs = "EPSG:4326"  # Common CRS (WGS 84)

# Function to set CRS if it's not set
def set_crs_if_needed(gdf, target_crs):
    if gdf.crs is None:
        gdf.set_crs(target_crs, inplace=True)
    return gdf

# Ensure all GeoDataFrames have the same CRS
local_roads = set_crs_if_needed(local_roads, target_crs)
main_roads = set_crs_if_needed(main_roads, target_crs)
villages = set_crs_if_needed(villages, target_crs)
protected_areas = set_crs_if_needed(protected_areas, target_crs)
waterways = set_crs_if_needed(waterways, target_crs)

# Convert geometries to centroids if they are not points
villages['geometry'] = villages['geometry'].centroid
local_roads['geometry'] = local_roads['geometry'].centroid
main_roads['geometry'] = main_roads['geometry'].centroid
protected_areas['geometry'] = protected_areas['geometry'].centroid
waterways['geometry'] = waterways['geometry'].centroid

# Transform all GeoDataFrames to the target CRS
local_roads = local_roads.to_crs(target_crs)
main_roads = main_roads.to_crs(target_crs)
villages = villages.to_crs(target_crs)
protected_areas = protected_areas.to_crs(target_crs)
waterways = waterways.to_crs(target_crs)

@jit(nopython=True)
def haversine(lon1, lat1, lon2, lat2):
    # convert decimal degrees to radians
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    
    # haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    
    # 6371 km is the radius of the Earth
    km = 6371 * c
    m = km * 1000
    return m

@jit(nopython=True)
def calculate_nearest_distance_numba(lons1, lats1, lons2, lats2):
    distances = np.empty(len(lons1))
    for i in range(len(lons1)):
        min_dist = np.inf  # Use np.inf instead of 'inf'
        lon1, lat1 = lons1[i], lats1[i]
        for j in range(len(lons2)):
            lon2, lat2 = lons2[j], lats2[j]
            dist = haversine(lon1, lat1, lon2, lat2)
            if dist < min_dist:
                min_dist = dist
        distances[i] = min_dist
    return distances

# Extract longitude and latitude arrays for mining locations and other features
mining_lons, mining_lats = mining_locations.geometry.x.values, mining_locations.geometry.y.values

village_lons, village_lats = villages.geometry.x.values, villages.geometry.y.values
waterway_lons, waterway_lats = waterways.geometry.x.values, waterways.geometry.y.values
local_road_lons, local_road_lats = local_roads.geometry.x.values, local_roads.geometry.y.values
main_road_lons, main_road_lats = main_roads.geometry.x.values, main_roads.geometry.y.values
protected_area_lons, protected_area_lats = protected_areas.geometry.x.values, protected_areas.geometry.y.values

# Calculate closest distances using Numba-accelerated function
mining_locations['distance_to_village'] = calculate_nearest_distance_numba(mining_lons, mining_lats, village_lons, village_lats)
mining_locations['distance_to_waterway'] = calculate_nearest_distance_numba(mining_lons, mining_lats, waterway_lons, waterway_lats)
mining_locations['distance_to_local_road'] = calculate_nearest_distance_numba(mining_lons, mining_lats, local_road_lons, local_road_lats)
mining_locations['distance_to_main_road'] = calculate_nearest_distance_numba(mining_lons, mining_lats, main_road_lons, main_road_lats)
mining_locations['distance_to_protected_area'] = calculate_nearest_distance_numba(mining_lons, mining_lats, protected_area_lons, protected_area_lats)

# Save the updated DataFrame to a new CSV file
mining_locations.to_csv('/home/kaiwen1/30123-Project-KaiwenNDVI-sample-with-distances.csv', index=False)

# Display the first few rows of the updated DataFrame
print(mining_locations.head())


/var/folders/66/zb_slx1j7jb2s05rx_q138bw0000gn/T/ipykernel_30828/283093530.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['geometry'] = df.apply(lambda row: Point(row['Longitude'], row['Latitude']), axis=1)
/var/folders/66/zb_slx1j7jb2s05rx_q138bw0000gn/T/ipykernel_30828/283093530.py:30: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  villages['geometry'] = villages['geometry'].centroid
/var/folders/66/zb_slx1j7jb2s05rx_q138bw0000gn/T/ipykernel_30828/283093530.py:31: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geome

   Unnamed: 0  Longitude  Latitude      NDVI   Band1   Band2   Band3    Band6  \
0           0  26.831496 -1.909149  0.343059  8556.0  8553.0  8941.0  11914.0   
1           1  26.831496 -1.909420  0.387653  8422.0  8491.0  9019.0  12541.0   
2           2  26.831765 -1.909420  0.357498  8573.0  8584.0  8971.0  11844.0   
3           3  26.832035 -1.909420  0.357075  8672.0  8687.0  9083.0  12086.0   
4           4  26.832305 -1.909420  0.376790  8651.0  8639.0  9041.0  12372.0   

    Band10                   geometry  distance_to_village  \
0  44125.0  POINT (26.83150 -1.90915)         43263.439539   
1  44126.0  POINT (26.83150 -1.90942)         43237.994071   
2  44144.0  POINT (26.83177 -1.90942)         43221.876826   
3  44152.0  POINT (26.83204 -1.90942)         43205.774369   
4  44175.0  POINT (26.83230 -1.90942)         43189.686716   

   distance_to_waterway  distance_to_local_road  distance_to_main_road  \
0           5962.628502            24011.785766           36014.73

In [6]:
print(villages.columns)
print(local_roads.columns)
print(main_roads.columns)
print(protected_areas.columns)
print(waterways.columns)

Index(['thematic', 'mkey', 'osmtype', 'osm_id', 'lat', 'lon', 'place',
       'abandoned:place', 'name', 'other_tags', 'geometry'],
      dtype='object')
Index(['thematic', 'mkey', 'osmtype', 'osm_id', 'lat', 'lon', 'name', 'ref',
       'highway', 'tunnel', 'bridge', 'junction', 'route', 'layer', 'oneway',
       'lanes', 'width', 'maxspeed', 'surface', 'practicability', 'smoothness',
       'condition', 'other_tags', 'geomtype', 'geometry'],
      dtype='object')
Index(['thematic', 'mkey', 'osmtype', 'osm_id', 'lat', 'lon', 'name', 'ref',
       'highway', 'tunnel', 'bridge', 'junction', 'route', 'layer', 'oneway',
       'lanes', 'width', 'maxspeed', 'surface', 'practicability', 'smoothness',
       'condition', 'other_tags', 'geomtype', 'geometry'],
      dtype='object')
Index(['WDPAID', 'WDPA_PID', 'PA_DEF', 'NAME', 'ORIG_NAME', 'DESIG',
       'DESIG_ENG', 'DESIG_TYPE', 'IUCN_CAT', 'INT_CRIT', 'MARINE',
       'REP_M_AREA', 'GIS_M_AREA', 'REP_AREA', 'GIS_AREA', 'NO_TAKE',
       